<font size=7 face="courier"> Source Code

This code creates a emailer bot that sends people warnings when devices that are part of live experiments fail.

# <font color="gray">setup notebook

import stuff

In [1]:
#!pip install ipydatetime

In [2]:
import smtplib as smtp
from email.mime.text import MIMEText
import ipywidgets as w
from IPython.core.display import HTML, display, Javascript, clear_output
import time
from braingeneers import iot
from  datetime import datetime,timedelta
import traceback
import schedule

In [3]:
#import ipydatetime

## <font color="gray"> Change Computer time zone

type these commands in the terminal to set the time zone. Solution from [here](https://dev.to/0xbf/set-timezone-in-your-docker-image-d22)

```
ln -fs /usr/share/zoneinfo/America/Los_Angeles /etc/localtime
dpkg-reconfigure -f noninteractive tzdata
```

# <font color="green">Gui App

## <font color="green"> widgets

we create all the necessary widgets for creating GUI that user uses to set up emailer bot

### <font color="green"> Email Recipeints

Where the use types who they want to have receive emails

In [4]:
w_emails_txt = w.HTML( """<p>Type the emails for people you want <code>BraingeneersBot</code> to notify. Use commas and no spaces! Example: <code>me@gmail.com,you@gmail.com</code></p>""" )
w_emails_txt
#<h3>Type Emails</h3>


HTML(value='<p>Type the emails for people you want <code>BraingeneersBot</code> to notify. Use commas and no s…

In [5]:
w_emails = w.Text(description="Emails",  layout= w.Layout(width='80%'))
w_emails

Text(value='', description='Emails', layout=Layout(width='80%'))

### <font color="green">Experiment Name

This is the name of the experimetn as it appears in emails

In [6]:
w_experiment = w.Text(description="Experiment",  layout= w.Layout(width='450px'))
w_experiment

Text(value='', description='Experiment', layout=Layout(width='450px'))

### <font color="green">IoT Devices

This is where the use types the names of the IoT devices that they want `braingeneersBot` to check on.

In [7]:
w_devices_txt = w.HTML( """<p>Choose the name of your experiment as it will appear in emails. Specify all the devices that are part of this experiment and and how often you want <code>BraingeneersBot</code> to check on them. For devices, use commas and no spaces! Example: <code>flow_1,pic_1</code></p>""" )
w_devices_txt


HTML(value='<p>Choose the name of your experiment as it will appear in emails. Specify all the devices that ar…

In [8]:
w_devices = w.Text(description="Devices",  layout= w.Layout(width='450px'))
w_devices

Text(value='', description='Devices', layout=Layout(width='450px'))

### <font color="green"> Check time

Where user speciefies how how much time much pass before `bot` sends warming. This is also how often the bot will send a warning message.

In [9]:
w_interval = w.IntText( value=2, description='Hours' )
w_interval

IntText(value=2, description='Hours')

### <font color="green"> Start/Stop Button

Button which starts `bot`

In [10]:
w_start_btn = w.Button( description="Start", button_style='success',  layout= w.Layout(width='auto') )
w_start_btn

Button(button_style='success', description='Start', layout=Layout(width='auto'), style=ButtonStyle())

Button that stops `bot`

In [11]:
w_stop_btn = w.Button( description="Stop", button_style='danger',  layout= w.Layout(width='auto') )
w_stop_btn

Button(button_style='danger', description='Stop', layout=Layout(width='auto'), style=ButtonStyle())

### <font color="green">Date/time picker

these widgets are used to set the stop time of `braingeneerbot`

In [12]:
w_date_txt = w.HTML( """<p>Select the final day and time of the experiment. <code>BraingeneerBot</code> will stop checking for device updates after this time has been reached. <code>BraingeneerBot</code> uses military time.</p>""" )



In [13]:
w_date = w.DatePicker()
w_date

DatePicker(value=None)

In [14]:
w_hour = w.IntSlider( value=7,description='hour', min=0, max=23)
w_hour 

IntSlider(value=7, description='hour', max=23)

In [15]:
w_minute = w.IntSlider( value=7,description='minute', min=0, max=59)
w_minute

IntSlider(value=7, description='minute', max=59)

## <font color="green">Widget Boxes

These boxes hold multiple widgets.

### <font color="green">Start Box

In [16]:
box_start = w.VBox([ w_emails_txt, w_emails, w_devices_txt, w_experiment, w_devices,w_interval,w_date_txt, w_date,w_hour,w_minute, w.HBox([w_start_btn])   ])
box_start


### <font color="green">?Stop Box?

# <font color="blue">Send Email

A helper function that to easily send emails.

In [17]:
def email(subject, message):
    s = smtp.SMTP_SSL('smtp.gmail.com', 465)
    s.login('braingeneerbot@gmail.com', 'injkvmljhfhjdutr')
    emails = w_emails.value.replace(" ", "").split(",") + ["braingeneerbot@gmail.com"]

    msg = MIMEText(message)
    msg['Subject'] = subject
    msg['From'] = 'braingeneerbot@gmail.com'
    msg['To'] = ", ".join(emails)

    s.sendmail('braingeneerbot@gmail.com', emails, msg.as_string())
    s.close()

In [18]:
#email("First Function", "this email was created with the function, email.")

# <font color="orange">Check IoT States

Checks if IoT devices are running properly. Sends error message if there are any problems.

In [19]:
def checkDevices():
    emails = w_emails.value.replace(" ", "").split(",") + ["braingeneerbot@gmail.com"]
    devices = w_devices.value.replace(" ", "").split(",")

    for device in devices:
        try: 
            history = iot.get_history(device)
            last_run = datetime.fromisoformat( history[-1][:19] )
            too_long = (datetime.now() - last_run) > timedelta( hours= w_interval.value )

            if too_long:
                message = f"""An IoT device has been offline for more than {w_interval.value} hours. This likely means that an error occured on the device and it is no longer running. This should be looked into immediately!

                Experiment: {w_experiment.value}
                Device: {device}"""
                email( "WARNING: IoT Experiment Device is Offline ", message )
                
        except Exception as e:
            message = f"""An error occured when checking the status of an IoT device. This likely means the device shutdown prematurely or hasn't started running. The device name could also be incorreclty typed into BraingeneerBot. This should be looked into immediately!

            Experiment: {w_experiment.value}
            Device: {device}
            Error:
            {traceback.format_exc()}"""
            email( "WARNING: IoT Experiment Device Status Check Error", message )
            
        

## <font color="brown"> Scratch Paper

# <font color="purple"> Set up Schedule

## <font color="brown">Scratch Paper

testit.today()

history[-1][:19]

testit = w_date.value

testit

testit + timedelta(hours=5)

 end_time= datetime.combine( w_date.value, datetime.min.time() ) + timedelta(hours=w_hour.value, minutes=w_minute.value)

w_minute.value

w_minute.value

end_time

# <font color="green">Start Function

This is the function that start the program when the user clicks the start button

In [20]:
def startBot(b):
    
    # Display new output
    clear_output()   
    display( w.HTML( """<p><code>BraingeneerBot</code> has been started! You should receive an email now confirming this. <code>BraingeneerBot</code> will continue to run until either the stop time of the experiment, or until you click the <code>Kernel->Restart & Clear Output</code> at the top of this page. Closing this window will not stop <code>BraingeneerBot</code> from continuing to run.</p>""") )

    # Send Email notfication to recipients
    end_time= datetime.combine( w_date.value, datetime.min.time() ) + timedelta(hours=w_hour.value, minutes=w_minute.value)
    message = f"""A new Braingeneers IoT experiment has been launched! BraingeneerBot will notify you if any devices have catastrophic issues. BraingeneerBot has been programmed to check on the experiment every {w_interval.value} hours. Please make sure any BraingeneerBot notifications are immediately addressed.

    Experiment: {w_experiment.value}
    Devices: {w_devices.value}
    End Time: {end_time.strftime("%m/%d/%Y, %H:%M") }"""
    email( "New IoT Experiment is Online", message )
    
    # Create Scheduled event and run it
    schedule.every( w_interval.value ).hours.do( checkDevices )
    while datetime.now() < end_time:
        schedule.run_pending()
        time.sleep(20)
        
w_start_btn.on_click( startBot )    
    

## <font color='brown'>Scratch Paper

## <font color="brown">scratch paper

# <font color="brown">Scratch Paper

## <font color="green">Test online code 1

[code from here](https://stackoverflow.com/questions/72480454/sending-email-with-python-google-disables-less-secure-apps)

## <font color="green">Test online code 2

[code from here](https://stackoverflow.com/questions/8856117/how-to-send-email-to-multiple-recipients-using-python-smtplib)